In [4]:
%load_ext autoreload
%autoreload 2

In [1]:
# handle gpu leakage issue on DSI cluster
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [11]:
from typing import List, Optional, Tuple, Union

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, LlamaForCausalLM, LlamaTokenizer
from transformers.modeling_outputs import CausalLMOutputWithPast

import scripts
from scripts import Linear

In [6]:
# Confirm GPUs are working properly and set default device explicitly
device = "cuda:0"
torch.cuda.device_count()

1

In [7]:
MODEL_DIR = "/net/projects/veitch/LLMs/"
MODEL = "llama1-based-models/alpaca-7b" # or llama2-based-models/llama2-hf/Llama-2-7b-chat-hf
MODEL_PATH = MODEL_DIR + MODEL

In [8]:
tokenizer = LlamaTokenizer.from_pretrained(MODEL_PATH)

You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [9]:
model = LlamaForCausalLM.from_pretrained(MODEL_PATH, low_cpu_mem_usage=True, torch_dtype=torch.float16, device_map="auto").to(device)

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [14]:
WEIGHTS_FOLDER = "model-weights/"
PROBE_FILE = "probe_weights.pt"

In [15]:
checkpoint = torch.load(WEIGHTS_FOLDER + PROBE_FILE)
config = checkpoint['config']
probe = Linear(config['hidden_size'], config['n_classes'])
probe.load_state_dict(checkpoint['state_dict'])
probe.eval()

Linear(
  (fc1): Linear(in_features=135168, out_features=12, bias=True)
)